In [1]:
import pandas as pd
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from typing import List
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
instructor.patch()

In [3]:
class ConversationObject(BaseModel):
    role: str
    content: str = Field('...', description = 'if the role is `assistant`, it must convert user content into function parameter format `<functioncall> {"name", "arguments"}`')
        
class JsonObject(BaseModel):
    function_call: str
    conversations: List[ConversationObject]

In [4]:
openai.api_type = "azure"
openai.api_base = "https://husein-ai5-aiservices1787699516.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''
engine = 'gpt-35-turbo'

In [5]:
def predict(prompt, max_tokens = 1024):
    messages=[
        {"role": "user", "content": prompt},
       
    ]
    try:
        response = openai.ChatCompletion.create(
        engine=engine,
        messages=messages,
        temperature=1.0,
        max_tokens=max_tokens,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
        )
        return response
    except Exception as e:
        print(e)
        return None
    
def predict_instruct(text):
    r = openai.ChatCompletion.create(
        engine=engine,
        response_model=JsonObject,
        messages=[
            {"role": "user", "content": text},
        ],
        temperature=1.0,
        max_tokens=1024,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return r

In [6]:
function_call = {
    "name": "KG",
    "description": "ekstrak dengan betul `KG` dengan semua parameter yang diperlukan dengan jenis yang betul",
    "parameters": {
        "$defs": {
            "Triplet": {
                "properties": {
                    "subject": {
                        "type": "string"
                    },
                    "predicate": {
                        "type": "string"
                    },
                    "object": {
                        "type": "string"
                    }
                },
                "required": [
                    "subject",
                    "predicate",
                    "object"
                ],
                "type": "object"
            }
        },
        "properties": {
            "triplets": {
                "items": {
                    "$ref": "#/$defs/Triplet"
                },
                "type": "array"
            }
        },
        "required": [
            "triplets"
        ],
        "type": "object"
    }
}

assistant = eval(
"""
{"name": "KG", "arguments": {"triplets": [{"subject": "Padah", "predicate": "mempunyai", "object": "hubungan sulit"}, {"subject": "hubungan sulit", "predicate": "dengan", "object": "pekerja sendiri"}, {"subject": "Padah", "predicate": "dipecat", "object": "CEO McDonald's"}]}}
"""
)
assistant = '<functioncall> ' + str(assistant)

In [7]:
conversations = [
    {
        'role': 'user',
        'content': "Padah jalin hubungan sulit dengan pekerja sendiri, CEO McDonald's dipecat serta merta"
    },
    {
        'role': 'assistant',
        'content': assistant,
    }
]

e = {
    'function_call': json.dumps(function_call),
    'conversations': conversations,
}
text = f"""Generate complex Malay function call prompts inspired by the given example. If translation is challenging, use Malaysian terms for variety. Maintain the JSON format, altering only key contents. Follow instructions precisely, avoiding randomness.

Code snippet for inspiration:
```
{e}
```
"""

In [8]:
r = predict_instruct(text)
r.dict()

/tmp/ipykernel_9570/2280122559.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  r.dict()


{'function_call': '{"name":"KG","description":"ekstrak dengan betul `KG` dengan semua parameter yang diperlukan dengan jenis yang betul","parameters":{"$defs":{"Triplet":{"properties":{"subject":{"type":"string"},"predicate":{"type":"string"},"object":{"type":"string"}},"required":["subject","predicate","object"],"type":"object"}},"properties":{"triplets":{"items":{"$ref":"#/$defs/Triplet"},"type":"array"},"required":["triplets"],"type":"object"}}',
 'conversations': [{'role': 'user',
   'content': 'Jadikanlah musim panas ini terasa lebih sejuk dengan meminum jus mango di tepi pantai yang indah ini'},
  {'role': 'assistant', 'content': '...'}]}

In [9]:
!mkdir function-call-complex-v2

mkdir: cannot create directory ‘function-call-complex-v2’: File exists


In [ ]:
from tqdm import tqdm

for i in tqdm(range(20000)):
    filename = f'function-call-complex-v2/{i}.json'
    if os.path.exists(filename):
        continue
    
    try:
        r = predict_instruct(text)
        with open(filename, 'w') as fopen:
            json.dump(r.dict(), fopen)
    except:
        pass

  0%|                                                                                                                   | 0/20000 [00:00<?, ?it/s]/tmp/ipykernel_9570/3318933309.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  json.dump(r.dict(), fopen)
 41%|██████████████████████████████████████████▌                                                             | 8177/20000 [09:57<14:41, 13.41it/s]